In [1]:
#--- get last rows of feature maps
# assume that we have (9x9) x (7x7) patches 

import numpy as np 

mat = np.random.randn(2,9,9,7,7)

In [2]:
mat.shape

(2, 9, 9, 7, 7)

In [3]:
# --- 
mat[:,8,:,:,:].shape


(2, 9, 7, 7)

In [4]:
x = np.arange(10)
x.shape = (10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
x[-4:-2]

array([6, 7])

In [6]:
x[-5:-2]

array([5, 6, 7])

In [7]:
# --- 
mat[:,8,:,:,:]

array([[[[-7.07788598e-01, -5.44850452e-01,  2.13587206e-01,
          -1.07198580e+00,  1.42356737e+00,  2.99860508e-01,
          -1.40759163e+00],
         [-8.80363910e-01, -1.67272542e+00, -1.85851511e+00,
           9.78887458e-01,  2.92878746e-01,  2.42465142e-01,
           8.77266122e-01],
         [-9.94688636e-02, -7.54988224e-01, -1.54263458e-01,
           2.75875210e-01, -1.24869174e+00,  1.09207802e+00,
           4.79760571e-01],
         [ 1.46750605e+00, -2.55464958e+00, -3.04096508e-01,
           2.98645429e-01,  8.03736519e-01, -1.02266003e-01,
           1.09793856e+00],
         [ 5.64860087e-01, -6.96230237e-01, -5.82047953e-01,
          -1.26031626e-01,  7.22748818e-01, -9.98831193e-01,
           6.57906379e-01],
         [ 8.63033873e-01,  1.88446399e+00, -4.11477936e-01,
           1.49298856e+00, -2.09803859e-01,  9.17804124e-01,
           7.11662504e-01],
         [ 5.51023651e-01, -8.15055509e-01,  1.07851928e+00,
          -2.77568225e+00,  7.95232409e

In [5]:
#----
# ref : https://github.com/berniwal/swin-transformer-pytorch/blob/master/example.py
#---
import torch
from swin_transformer_pytorch import SwinTransformer

net = SwinTransformer(
    hidden_dim=96,
    layers=(2, 2, 6, 2),
    heads=(3, 6, 12, 24),
    channels=3,
    num_classes=3,
    head_dim=32,
    window_size=7,
    downscaling_factors=(4, 2, 2, 2),
    relative_pos_embedding=True
)

dummy_x = torch.randn(1, 3, 224, 224)
logits = net(dummy_x)  # (1,3)
print(net)
print(logits)

ModuleNotFoundError: No module named 'swin_transformer_pytorch'

In [22]:
import math
x = dummy_x = torch.randn(1, 3, 224, 224)
height = width = int(math.sqrt(x.shape[2])/2)

In [27]:
math.sqrt(224)/2

7.483314773547883

In [25]:
width

7

In [6]:
import torch
from torch import nn, einsum
import numpy as np
from einops import rearrange, repeat


class CyclicShift(nn.Module):
    def __init__(self, displacement):
        super().__init__()
        self.displacement = displacement

    def forward(self, x):
        return torch.roll(x, shifts=(self.displacement, self.displacement), dims=(1, 2))


In [9]:
def create_mask(window_size, displacement, upper_lower, left_right):
    mask = torch.zeros(window_size ** 2, window_size ** 2)

    if upper_lower:
        mask[-displacement * window_size:, :-displacement * window_size] = float('-inf')
        mask[:-displacement * window_size, -displacement * window_size:] = float('-inf')

    if left_right:
        mask = rearrange(mask, '(h1 w1) (h2 w2) -> h1 w1 h2 w2', h1=window_size, h2=window_size)
        mask[:, -displacement:, :, :-displacement] = float('-inf')
        mask[:, :-displacement, :, -displacement:] = float('-inf')
        mask = rearrange(mask, 'h1 w1 h2 w2 -> (h1 w1) (h2 w2)')

    return mask

In [8]:
window_size = 7 
displacement = 3

In [10]:
mask = torch.zeros(window_size**2,window_size**2 )

In [12]:
mask

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
mask[-displacement * window_size:, :-displacement * window_size] = float('-inf')
mask[:-displacement * window_size, -displacement * window_size:] = float('-inf')

In [15]:
mask

tensor([[0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.]])

25

In [ ]:
#--

window_size = 7 
displacement = 3 

#---
displacement = window_size // 2
self.cyclic_shift = CyclicShift(-displacement)
self.cyclic_back_shift = CyclicShift(displacement)
self.upper_lower_mask = nn.Parameter(create_mask(window_size=window_size, displacement=displacement,
                                                    upper_lower=True, left_right=False), requires_grad=False)
self.left_right_mask = nn.Parameter(create_mask(window_size=window_size, displacement=displacement,